<a href="https://colab.research.google.com/github/OmarGhoz/FreeCodeCamp-Machine-Learning-Projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-11-28 20:18:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M   161MB/s    in 0.2s    

2024-11-28 20:18:58 (161 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
# Filter the ratings dataset (This is where df_ratings_filtered was defined)
min_ratings = 10
book_counts = df_ratings.groupby('isbn').size()
popular_books = book_counts[book_counts >= min_ratings].index
df_ratings_filtered = df_ratings[df_ratings['isbn'].isin(popular_books)]

In [ ]:
# Convert the ratings data into a pivot table with users as rows, books as columns, and ratings as values
df_ratings_matrix = df_ratings_filtered.pivot(index='user', columns='isbn', values='rating').fillna(0)

In [ ]:
# Convert the DataFrame into a sparse matrix to save memory, as most ratings are 0
df_ratings_matrix_sparse = csr_matrix(df_ratings_matrix.values)

In [ ]:
# Function to return recommended books - this will be tested
def get_recommends(book=""):
    # Step 1: Create a NearestNeighbors model to find similar books
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1)

    # Fit the model on the sparse ratings matrix
    model_knn.fit(df_ratings_matrix_sparse)

    # Find the nearest neighbors for the given book
    book_index = df_books[df_books['title'] == book].index[0]
    distances, indices = model_knn.kneighbors(df_ratings_matrix_sparse[book_index], n_neighbors=6)

    # Step 2: Get the names of the recommended books
    recommended_books = [df_books.iloc[indices.flatten()[i]]['title'] for i in range(1, 6)]

    # Step 3: Return the book title along with the recommended books and their distances
    return book, [(recommended_books[i], distances.flatten()[i+1]) for i in range(5)]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()